In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import itertools as it
import datetime
import math

df = pd.read_csv('data_for_glicko.csv', low_memory = False)

In [18]:
#df = df[(df['Year'] == 2017)]
df = df[(df['Stage'] == -1)]
pd.to_datetime(df.Date)
df['Year'] = 0
df['Month'] = 0
df['Day'] = 0

df.Year = pd.DatetimeIndex(df.Date).year
df.Month = pd.DatetimeIndex(df.Date).month
df.Day = pd.DatetimeIndex(df.Date).day

# create day, month, year cols
df.head(10)

,Unnamed: 0,Rnk,Rider,Team,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,Year,Month,Day
15,15.0,53,1.0,1,27.0,5.0,5,-1.0,6/12/2011,-,0.0,0 km,12.0,2011,6,12
36,36.0,88,1.0,1,27.0,25.0,6,-1.0,7/24/2011,39.788 km/h,0.0,0 km,1.0,2011,7,24
58,58.0,126,1.0,1,27.0,20.0,8,-1.0,9/11/2011,-,0.0,0 km,6.0,2011,9,11
67,67.0,105,1.0,1,28.0,5.0,10,-1.0,1/22/2012,38.661 km/h,0.0,0 km,32.0,2012,1,22
74,74.0,85,1.0,1,28.0,5.0,11,-1.0,3/11/2012,40.862 km/h,0.0,0 km,9.0,2012,3,11
90,90.0,90,1.0,1,28.0,5.0,5,-1.0,6/10/2012,39.419 km/h,0.0,0 km,9.0,2012,6,10
103,103.0,44,1.0,1,28.0,5.0,17,-1.0,8/12/2012,41.869 km/h,0.0,0 km,29.0,2012,8,12
124,124.0,132,1.0,1,28.0,20.0,8,-1.0,9/9/2012,38.711 km/h,0.0,0 km,17.0,2012,9,9
133,133.0,104,1.0,1,29.0,5.0,11,-1.0,3/10/2013,40.604 km/h,0.0,0 km,14.0,2013,3,10
140,140.0,112,1.0,1,29.0,5.0,1,-1.0,3/24/2013,40.467 km/h,0.0,0 km,24.0,2013,3,24


In [5]:
##### RATE FUNCTION DON'T TOUCH!!!! ######
Q = math.log(10) / 400
def reduce_impact(rd):
        """The original form is `g(RD)`. This function reduces the impact of
        games as a function of an opponent's RD.
        """
        return (1 + (3 * (Q ** 2) * rd ** 2) / math.pi ** 2) ** -0.5

def expect_score(elo, other_elo, other_impact):
        return 1 / (1 + 10 ** (other_impact * (elo - other_elo) / -400.))

def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros
def onelistmaker(n):
    listofones = [1] * n
    return listofones
# elos = list(combined['old_elo'])
# rds = list(combined['old_rd'])
def rate(old_elos, old_rds): # assuming sorted by rank
        d_square_inv = 0
        difference = 0
        new_elos = []
        new_rds = []

        for i in range(len(old_elos)): # do this for every rider, i denotes rider that's being ranked
            if i < len(old_elos):
                d_square_inv = 0
                difference = 0
                denom = 0


                better = old_elos[:i]
                worse = old_elos[i+1:]
                other_elos = better + worse
                # now create a list of e.g. 3x0, rest ones if score 4th place, j denotes every other rider
                actual_score = zerolistmaker(len(better)) + onelistmaker(len(worse))
                for j in range(len(other_elos)):
                    impact = reduce_impact(other_elos[j])
                    expected_score = expect_score(old_elos[i], other_elos[j], impact)
                    difference += impact * (actual_score[j] - expected_score)
                    d_square_inv += (expected_score * (1 - expected_score) * (Q ** 2) * (impact ** 2))

                denom = 1/(old_rds[i] ** 2) + d_square_inv
                mu = old_elos[i]  + Q / denom * difference
                sigma1 = math.sqrt(1. / denom)
                sigma = min(math.sqrt(sigma1 ** 2 + 63.2 ** 2), 350)

                new_elos.append(mu)
                new_rds.append(sigma)
            else:
                d_square_inv = 0
                difference = 0
                denom = 0

                better = old_elos
                worse = []
                other_elos = better + worse
                # now create a list of e.g. 3x0, rest ones if score 4th place, j denotes every other rider
                actual_score = zerolistmaker(len(better)) + onelistmaker(len(worse))
                for j in range(len(other_elos)):
                    impact = reduce_impact(other_elos[j])
                    expected_score = expect_score(old_elos[i], other_elos[j], impact)
                    difference += impact * (actual_score[j] - expected_score)
                    d_square_inv += (expected_score * (1 - expected_score) * (Q ** 2) * (impact ** 2))
                denom = 1/(old_rds[i] ** 2) + d_square_inv
                mu = old_elos[i]  + Q / denom * difference
                sigma1 = math.sqrt(1. / denom)
                sigma = min(math.sqrt(sigma1 ** 2 + 63.2 ** 2), 350)
                new_elos.append(mu)
                new_rds.append(sigma)


        
        return(new_elos, new_rds)

In [8]:
# ONE GC SORTED BASED ON RANK
dat = df[(df['Stage'] == -1)].copy()
dat = dat[(dat['Name'] == "11.00")].sort_values("Pnt", ascending=False) # RNK 0 == DNS/DNS
#dat = dat[(dat['Rnk'] != 0.0)]
dat['old_elo'] = 1500
dat['new_elo'] = 0
dat['old_rd'] = 350
dat['new_rd'] = 0

Q = math.log(10) / 400
#dat['g_rd'] = (1 + (3 * (Q ** 2) * dat['old_rd'] ** 2) / math.pi ** 2) ** -0.5
dat.head(20)

,Unnamed: 0,Rnk,Rider,Team,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,old_elo,new_elo,old_rd,new_rd


In [5]:
# FILTER 2 BEST RANKED RIDERS
first = dat.groupby('Team').nth(0)
second = dat.groupby('Team').nth(1)

combined = first.append(second)
combined = combined.reset_index().sort_values('Pnt', ascending=False)
rada = np.arange(1, (len(combined.Rnk)+1))
combined.Rnk = rada
combined.head(20)

,Team,Unnamed: 0,Rnk,Rider,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd
11,58.0,18043,1,80.0,29.0,250.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
1,7.0,204503,2,1270.0,34.0,190.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
3,19.0,10864,3,55.0,30.0,160.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
0,1.0,31378,4,140.0,29.0,140.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
25,19.0,101165,5,491.0,24.0,120.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
15,75.0,41516,6,190.0,27.0,110.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
18,95.0,26447,7,121.0,28.0,100.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
19,103.0,120903,8,582.0,25.0,90.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
5,28.0,113956,9,552.0,24.0,80.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
10,52.0,114485,10,554.0,28.0,70.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0


In [6]:
result = rate(list(combined.old_elo), list(combined.old_rd))
combined.new_elo = result[0]
combined.new_rd = result[1]

In [7]:
combined.head(10)

,Team,Unnamed: 0,Rnk,Rider,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd
11,58.0,18043,1,80.0,29.0,250.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2595.844959,350,216.946151
1,7.0,204503,2,1270.0,34.0,190.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2544.875426,350,216.946151
3,19.0,10864,3,55.0,30.0,160.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2493.905893,350,216.946151
0,1.0,31378,4,140.0,29.0,140.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2442.936360,350,216.946151
25,19.0,101165,5,491.0,24.0,120.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2391.966827,350,216.946151
15,75.0,41516,6,190.0,27.0,110.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2340.997294,350,216.946151
18,95.0,26447,7,121.0,28.0,100.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2290.027761,350,216.946151
19,103.0,120903,8,582.0,25.0,90.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2239.058228,350,216.946151
5,28.0,113956,9,552.0,24.0,80.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2188.088695,350,216.946151
10,52.0,114485,10,554.0,28.0,70.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2137.119162,350,216.946151


In [8]:
dat['Date'] =  pd.to_datetime(dat['Date'])
dat[(dat['Stage'] == -1) & (dat['Name'] == "11.00") & (dat['Date'] >= pd.Timestamp(2017,3,10))].head(10)

,Unnamed: 0,Rnk,Rider,Team,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd
2082,18043,8.0,80.0,58.0,29.0,250.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
29568,204503,1.0,1270.0,7.0,34.0,190.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
1274,10864,3.0,55.0,19.0,30.0,160.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
3946,31378,49.0,140.0,1.0,29.0,140.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
13967,101165,41.0,491.0,19.0,24.0,120.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
5293,41516,5.0,190.0,75.0,27.0,110.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
3213,26447,42.0,121.0,95.0,28.0,100.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
16859,120903,47.0,582.0,103.0,25.0,90.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
15937,113956,32.0,552.0,28.0,24.0,80.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
16013,114485,178.0,554.0,52.0,28.0,70.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0


In [9]:
combined.Year[0]

2017

In [19]:
# CREATING THE LOOP
# 1. TAKE ONLY STAGE == -1
# 2. sort by date
# 3. rotate over all names - for name in table.name.unique() - make sure it's taking it in the correct order
# 4. take subset (1 race), compute new elo
# 5. update elos in the central table
for year in df.Year.unique():
    year11 = year

    df2 = df[(df['Stage'] == -1) & (df['Year'] == year11)] # df is already only 2017 for now
    df2 = df2.sort_values('Date', ascending = True)
    df2['old_elo'] = 1500
    df2['new_elo'] = 0
    df2['old_rd'] = 350
    df2['new_rd'] = 0
    date1 = []
    date = list(df2.Date)
    for i in range(len(df2.Date)):
        date1.append(pd.Timestamp(date[i]))
    df2['Date1'] = 0
    df2['Date1'] = date1

    # CREATE A LOOP EVALUATING EVERY RACE
    for j in range(len(df2.Name.unique())):
        race_name = list(df2.sort_values('Date',  ascending = True).Name.unique())[j]
        race_data = df2[(df2['Name'] == race_name)].sort_values('Pnt', ascending=False)
        # take only two best riders!!!!
        # FILTER 2 BEST RANKED RIDERS
        first = race_data.groupby('Team').nth(0)
        second = race_data.groupby('Team').nth(1)

        combined = first.append(second)
        combined = combined.reset_index().sort_values('Pnt', ascending=False)
        rada = np.arange(1, (len(combined.Rnk)+1))
        combined.Rnk = rada
        race_data = combined
        result = rate(list(race_data.old_elo), list(race_data.old_rd)) # compute new elos

    # create a empty df with all the cols we want
        #empty = pd.DataFrame(columns = df2.columns)

    # UPDATING OF VALUES, do after every race, has to be intended
        for i in range(len(race_data.Rider)): 
            rider = list(race_data.Rider)[i]
            elo = result[0][i]
            rd = result[1][i]
            date = pd.Timestamp(list(race_data.Year)[0], list(race_data.Month)[0], list(race_data.Day)[0])

            df2.loc[(df2['Rider'] == rider) & (df2['Date1'] > date), 'old_elo'] = elo
            df2.loc[(df2['Rider'] == rider) & (df2['Date1'] > date), 'old_rd'] = rd
            df2.loc[(df2['Rider'] == rider) & (df2['Date1'] == date), 'new_elo'] = elo
            df2.loc[(df2['Rider'] == rider) & (df2['Date1'] == date), 'new_rd'] = rd
        
        df2.loc[(df2['new_elo'] == 0), 'new_elo'] = df2.loc[(df2['new_elo'] == 0), 'old_elo']
        df2.loc[(df2['new_rd'] == 0), 'new_rd'] = df2.loc[(df2['new_rd'] == 0), 'old_rd']


        #   subset1 = df2[(df2['Rider'] == rider) & (df2['Date1'] > date)] # slice of df2 for every rider after this race
        #  subset1.loc[:,'old_elo']  = elo
        # subset1.loc[:,'old_rd']  = rd

            #subset2 = df2[(df2['Rider'] == rider) & (df2['Date1'] == date)] # slice of df2 for every rider after this race
            #subset2.loc[:,'new_elo']  = elo
            #subset2.loc[:,'new_rd']  = rd

            #subset3 = df2[(df2['Rider'] == rider) & (df2['Date1'] < date)]
            # and merge them all together afterwards (append), then use this df for next race
            #empty = empty.append(subset1)
            #empty = empty.append(subset2)
            #empty = empty.append(subset3)

            # HERE APPEND sets_merged TO EMPTY

        #df2 = empty
    df2.to_csv('results'+str(year11)+'_final.csv')

In [54]:
df2[(df2['Rider'] == 210)].sort_values('Date1', ascending=False)

,Unnamed: 0,Rnk,Rider,Team,Age,Pnt,Name,Stage,Date,Avg. speed winner,...,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd,Date1
5884,46024,45.0,210.0,41.0,32.0,3.0,63.00,-1.0,10/24/2017,-,...,0 km,46,2017,10,24,2562.523344,1500.000000,168.506369,350.000000,2017-10-24
5862,46002,18.0,210.0,41.0,32.0,250.0,5.00,-1.0,6/11/2017,-,...,0 km,6,2017,6,11,2203.144692,2562.523344,180.912632,168.506369,2017-06-11
5849,45989,107.0,210.0,41.0,32.0,5.0,66.00,-1.0,4/8/2017,-,...,0 km,24,2017,4,8,2203.144692,1500.000000,180.912632,350.000000,2017-04-08
5843,45983,80.0,210.0,41.0,32.0,40.0,1.00,-1.0,3/26/2017,-,...,0 km,18,2017,3,26,2070.491320,2203.144692,220.096791,180.912632,2017-03-26
5835,45975,26.0,210.0,41.0,31.0,55.0,11.00,-1.0,3/12/2017,-,...,0 km,4,2017,3,12,1500.000000,2070.491320,350.000000,220.096791,2017-03-12


In [12]:
date1 = []
date = list(df2.Date)
for i in range(len(df2.Date)):
    date1.append(pd.Timestamp(date[i]))
df2['Date1'] = 0
df2['Date1'] = date1

In [36]:
test = df2.loc[(df2['new_elo'] == 0)]
test.new_elo = test.old_elo
test

C:\Users\Tom\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Unnamed: 0,Rnk,Rider,Team,Age,Pnt,Name,Stage,Date,Avg. speed winner,...,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd,Date1
3247,26848,121.0,122.0,41.0,24.0,5.0,10.00,-1.0,1/22/2017,-,...,0 km,34,2017,1,22,1500.000000,1500.000000,350.000000,0.0,2017-01-22
18313,132090,122.0,633.0,75.0,28.0,5.0,10.00,-1.0,1/22/2017,-,...,0 km,34,2017,1,22,1500.000000,1500.000000,350.000000,0.0,2017-01-22
18110,130864,101.0,625.0,69.0,25.0,5.0,10.00,-1.0,1/22/2017,-,...,0 km,34,2017,1,22,1500.000000,1500.000000,350.000000,0.0,2017-01-22
17344,124844,51.0,594.0,105.0,29.0,5.0,10.00,-1.0,1/22/2017,-,...,0 km,34,2017,1,22,1500.000000,1500.000000,350.000000,0.0,2017-01-22
3572,28713,124.0,129.0,1.0,23.0,5.0,10.00,-1.0,1/22/2017,-,...,0 km,34,2017,1,22,1500.000000,1500.000000,350.000000,0.0,2017-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15610,111589,98.0,539.0,41.0,25.0,20.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,1500.000000,1500.000000,350.000000,0.0,2017-09-10
16004,114023,13.0,552.0,28.0,25.0,20.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,2363.823366,2363.823366,164.731304,0.0,2017-09-10
1710,14486,144.0,66.0,75.0,32.0,20.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,634.717779,634.717779,233.029349,0.0,2017-09-10
9674,73435,122.0,326.0,58.0,36.0,20.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,1500.000000,1500.000000,350.000000,0.0,2017-09-10


In [50]:
range(len(df2.Name.unique()))

range(0, 17)

In [30]:
race_data.sort_values("Pnt", ascending=False)

,Team,Unnamed: 0,Rnk,Rider,Age,Pnt,Name,Stage,Date,Avg. speed winner,...,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd,Date1
12,58.0,6815,13,38.0,32.0,400.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,2451.992748,0.0,167.690232,0.0,2017-09-10
17,95.0,42695,18,198.0,32.0,290.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,1502.758989,0.0,153.579058,0.0,2017-09-10
15,75.0,41578,16,190.0,27.0,240.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,2192.924003,0.0,155.508829,0.0,2017-09-10
18,103.0,46982,19,213.0,26.0,220.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,1717.698092,0.0,168.692398,0.0,2017-09-10
1,7.0,204569,2,1270.0,34.0,200.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,3115.943289,0.0,162.698041,0.0,2017-09-10
34,58.0,11656,35,57.0,29.0,190.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,2067.609547,0.0,221.328279,0.0,2017-09-10
13,69.0,28005,14,127.0,30.0,180.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,1132.984536,0.0,149.543516,0.0,2017-09-10
8,41.0,129864,9,619.0,23.0,170.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,1500.000000,0.0,350.000000,0.0,2017-09-10
10,46.0,109330,11,528.0,30.0,160.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,2314.189562,0.0,187.519109,0.0,2017-09-10
19,105.0,18723,20,82.0,29.0,150.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,1289.880076,0.0,162.663583,0.0,2017-09-10


In [59]:
df.Year.unique()

array([2017], dtype=int64)